In [1]:
from oo_cqed_rhf import CQEDRHFCalculator
import numpy as np
import psi4

In [2]:
lambda_vector = [0.0, 0.0, 0.05]
# molecule string for H2O
molecule_string = """

0 1
    O      0.000000000000   0.000000000000  -0.068516219320
    H      0.000000000000  -0.790689573744   0.543701060715
    H      0.000000000000   0.790689573744   0.543701060715
no_reorient
no_com
symmetry c1
"""

# options for H2O
psi4_options = {
    "basis": "cc-pVDZ",
    "save_jk": True,
    "scf_type": "pk",
    "e_convergence": 1e-12,
    "d_convergence": 1e-12,
}


In [5]:
calc = CQEDRHFCalculator(lambda_vector, molecule_string, psi4_options)
calc.calc_cqed_rhf_energy()


Scratch directory: /tmp/
   => Libint2 <=

    Primary   basis highest AM E, G, H:  5, 4, 3
    Auxiliary basis highest AM E, G, H:  6, 5, 4
    Onebody   basis highest AM E, G, H:  6, 5, 4
    Solid Harmonics ordering:            gaussian

*** tstart() called on CHEM9LYY31VAWS
*** at Mon Jun 16 13:50:44 2025

   => Loading Basis Set <=

    Name: CC-PVDZ
    Role: ORBITAL
    Keyword: BASIS
    atoms 1   entry O          line   198 file /Users/jfoley19/Code/psi4/objdir-Release/stage/share/psi4/basis/cc-pvdz.gbs 
    atoms 2-3 entry H          line    22 file /Users/jfoley19/Code/psi4/objdir-Release/stage/share/psi4/basis/cc-pvdz.gbs 


         ---------------------------------------------------------
                                   SCF
               by Justin Turney, Rob Parrish, Andy Simmonett
                          and Daniel G. A. Smith
                              RHF Reference
                        1 Threads,    500 MiB Core
         ----------------------------------

In [7]:
expected_h2o_e = -76.016355284146
assert np.isclose(expected_h2o_e, calc.cqed_rhf_energy)

In [8]:
calc.calc_scf_gradient(qed_wfn=True)


*** tstart() called on CHEM9LYY31VAWS
*** at Mon Jun 16 13:50:45 2025


         ------------------------------------------------------------
                                   SCF GRAD                          
                          Rob Parrish, Justin Turney,                
                       Andy Simmonett, and Alex Sokolov              
         ------------------------------------------------------------

  ==> Geometry <==

    Molecular point group: c1
    Full point group: C2v

    Geometry (in Angstrom), charge = 0, multiplicity = 1:

       Center              X                  Y                   Z               Mass       
    ------------   -----------------  -----------------  -----------------  -----------------
         O            0.000000000000     0.000000000000    -0.068516219320    15.994914619570
         H            0.000000000000    -0.790689573744     0.543701060715     1.007825032230
         H            0.000000000000     0.790689573744     0.54

array([[-3.18324750e-16, -1.92665585e-15, -6.01300796e-02],
       [-1.31583620e-16, -4.33923429e-02,  3.00650398e-02],
       [ 4.49908370e-16,  4.33923429e-02,  3.00650398e-02]])

r repulsion =    8.801465564563406

  ==> Basis Set <==

  Basis Set: CC-PVDZ
    Blend: CC-PVDZ
    Number of shells: 12
    Number of basis functions: 24
    Number of Cartesian functions: 25
    Spherical Harmonics?: true
    Max angular momentum: 2

  ==> DirectJKGrad: Integral-Direct SCF Gradients <==

    Gradient:                    1
    J tasked:                  Yes
    K tasked:                  Yes
    wK tasked:                  No
    Integrals threads:           1
    Schwarz Cutoff:          1E-12


  -Total Gradient:
     Atom            X                  Y                   Z
    ------   -----------------  -----------------  -----------------
       1       -0.000000000000    -0.000000000000    -0.060130079561
       2       -0.000000000000    -0.043392342878     0.030065039780
       3        0.000000000000     0.043392342878     0.030065039780


*** tstop() called on CHEM9LYY31VAWS at Mon Jun 16 13:50:45 2025
Module time:
	user time   =       0.04 seconds =       

In [ ]:
mol = psi4.geometry(calc.molecule_string)
psi4.set_options(calc.psi4_options)
gradient = np.asarray(psi4.gradient("scf"))


In [ ]:
print(calc.scf_grad -gradient)

In [ ]:
calc.calc_quadrupole_gradient()
# last output

#[[ 1.32601432e-18  9.99294165e-19 -2.77573793e-03]
# [ 1.03219418e-19  2.17498212e-04  1.62681438e-03]
# [-1.42923374e-18 -2.17498212e-04  1.62681438e-03]]

In [ ]:
calc.calc_dipole_dipole_gradient()

In [ ]:
print(calc.K_dse_gradient.reshape(3,3))

In [ ]:
total_gradient = calc.scf_grad + calc.K_dse_gradient.reshape(3,3) + calc.o_dse_gradient.reshape(3,3)
print(total_gradient)

# The numerical gradient for water in a cc-pVDZ basis set with this geometry and lambda = 0, 0, 0.05 is:

[[ 2.25601351e-10 -2.63201576e-10 -6.18353559e-02]

 [ 7.52004504e-11 -4.35898538e-02  3.09176777e-02]
 
 [ 0.00000000e+00  4.35898541e-02  3.09176786e-02]]

In [ ]:
_expected_gradient = np.array(
[[ 2.25601351e-10, -2.63201576e-10, -6.18353559e-02],
 [ 7.52004504e-11, -4.35898538e-02,  3.09176777e-02],
 [ 0.00000000e+00,  4.35898541e-02,  3.09176786e-02]])

print(_expected_gradient)